In [1]:
import torch
import plotly.graph_objects as go
import numpy as np

from _src import ModelConfig
from _src import GaussianProcess, tStudentProcess

In [3]:
class BayesianOptimizer:
    def __init__(self, surrogate, acquisition, bounds, objective_func):
        """
        Bayesian Optimization class.

        Parameters
        ----------
        surrogate: instance of GaussianProcess or tStudentProcess
            Surrogate model.
        acquisition: instance of Acquisition
            Acquisition function.
        bounds: dict
            Dictionary specifying parameter bounds, keys are parameter names, values are tuples (min, max).
        objective_func: callable
            The objective function to optimize.
        """
        self.surrogate = surrogate
        self.acquisition = acquisition
        self.bounds = bounds
        self.objective_func = objective_func

        self.X = None  # Observations
        self.y = None  # Observed function values
        self.tau = None  # Best observed function value
        self.history = []

    def initialize(self, n_init=3):
        """
        Performs initial evaluations before fitting the surrogate model.
        """
        X_init = []
        y_init = []
        for _ in range(n_init):
            x = self.sample_random_point()
            y = self.objective_func(**x)
            X_init.append(list(x.values()))
            y_init.append(y)
        self.X = torch.tensor(
            X_init, dtype=self.surrogate.config.device.dtype, device=self.surrogate.config.device.device
        )
        self.y = torch.tensor(
            y_init, dtype=self.surrogate.config.device.dtype, device=self.surrogate.config.device.device
        )
        self.surrogate.fit(self.X, self.y)
        self.tau = torch.max(self.y).item()
        self.history.append(self.tau)

    def sample_random_point(self):
        """
        Samples a random point within the bounds.

        Returns
        -------
        dict
            Randomly sampled point as a dictionary of parameter values.
        """
        x = {}
        for param_name, (lower, upper) in self.bounds.items():
            x[param_name] = np.random.uniform(lower, upper)
        return x

    def acquisition_function_wrapper(self, x_tensor):
        """
        Wrapper for the acquisition function for optimization.

        Parameters
        ----------
        x_tensor: torch.Tensor
            Input tensor for the acquisition function.

        Returns
        -------
        torch.Tensor
            Negative acquisition function value.
        """
        x = x_tensor.unsqueeze(0)  # Add batch dimension
        mean, cov = self.surrogate.predict(x, return_std=True)
        std = torch.sqrt(cov + 1e-6)
        acq_value = self.acquisition.eval(self.tau, mean, std)
        return -acq_value

    def optimize_acquisition(self, n_restarts=5, lr=0.01, n_iter=50):
        """
        Optimizes the acquisition function to find the next point to evaluate.

        Parameters
        ----------
        n_restarts: int
            Number of random starting points for optimization.
        lr: float
            Learning rate for the optimizer.
        n_iter: int
            Number of iterations for the optimizer.

        Returns
        -------
        dict
            Next point to evaluate.
        """
        best_acq_value = float('inf')
        best_x = None

        bounds_tensor = torch.tensor(
            [self.bounds[param_name] for param_name in self.bounds.keys()],
            dtype=self.surrogate.config.device.dtype,
            device=self.surrogate.config.device.device,
        )

        for _ in range(n_restarts):
            # Initialize x within bounds
            x0_dict = self.sample_random_point()
            x0_list = [x0_dict[param_name] for param_name in self.bounds.keys()]
            x = torch.tensor(
                x0_list,
                dtype=self.surrogate.config.device.dtype,
                device=self.surrogate.config.device.device,
                requires_grad=True,
            )

            optimizer = torch.optim.Adam([x], lr=lr)

            for _ in range(n_iter):
                optimizer.zero_grad()
                acq_value = self.acquisition_function_wrapper(x)
                acq_value.backward()
                optimizer.step()

                # Project back into bounds
                with torch.no_grad():
                    for i, param_name in enumerate(self.bounds.keys()):
                        lower, upper = self.bounds[param_name]
                        x.data[i] = torch.clamp(x.data[i], lower, upper)

            final_acq_value = self.acquisition_function_wrapper(x).item()
            if final_acq_value < best_acq_value:
                best_acq_value = final_acq_value
                best_x = x.detach().clone()

        x_next = {param_name: best_x[i].item() for i, param_name in enumerate(self.bounds.keys())}

        return x_next
    
    def update_model(self, x_new_dict, y_new):
        """
        Updates the surrogate model with the new data point.
        """
        x_new = torch.tensor(
            [list(x_new_dict.values())], dtype=self.surrogate.config.device.dtype, device=self.surrogate.config.device.device
        )
        y_new = torch.tensor(
            [y_new], dtype=self.surrogate.config.device.dtype, device=self.surrogate.config.device.device
        )
        if self.X is None:
            self.X = x_new
            self.y = y_new
        else:
            self.X = torch.cat((self.X, x_new), dim=0)
            self.y = torch.cat((self.y, y_new), dim=0)
        self.surrogate.fit(self.X, self.y)
        self.tau = torch.max(self.y).item()
        self.history.append(self.tau)

    def run(self, n_iterations=10, n_init=3, n_restarts=5):
        """
        Runs the Bayesian optimization process.

        Parameters
        ----------
        n_iterations: int
            Number of iterations to run.
        n_init: int
            Number of initial evaluations.
        n_restarts: int
            Number of random restarts in acquisition optimization.
        """
        if self.X is None:
            self.initialize(n_init=n_init)

        for _ in range(n_iterations):
            x_next = self.optimize_acquisition(n_restarts=n_restarts)
            y_next = self.objective_func(**x_next)
            self.update_model(x_next, y_next)

    def get_best(self):
        """
        Returns the best observed point and its function value.

        Returns
        -------
        dict
            Best point as a dictionary of parameter values.
        float
            Best observed function value.
        """
        best_index = torch.argmax(self.y).item()
        best_x_array = self.X[best_index].cpu().numpy()
        best_x = {param_name: best_x_array[i] for i, param_name in enumerate(self.bounds.keys())}
        best_y = self.y[best_index].item()
        return best_x, best_y

In [4]:
# Define your objective function
def objective_func(x):
    # Example: simple quadratic function
    return - (x['x'] - 2) ** 2 + 3

# Define bounds for your parameters
bounds = {'x': (0, 5)}

# Create a ModelConfig instance (you may need to set appropriate values)
config = ModelConfig(ard_num_dims=1)

# Choose a surrogate model (GaussianProcess or tStudentProcess)
surrogate = GaussianProcess(config)

# Choose an acquisition function
acquisition = Acquisition(mode='ExpectedImprovement')

# Create the Bayesian optimizer
optimizer = BayesianOptimizer(surrogate, acquisition, bounds, objective_func)

# Run the optimization
optimizer.run(n_iterations=10)

# Get the best result
best_x, best_y = optimizer.get_best()
print(f"Best x: {best_x}, Best y: {best_y}")

/Users/keisukeonoue/ws/pyGPGO/src/covfunc.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(config.priors.noise_prior.mean, device=config.device.device, dtype=config.device.dtype)


TypeError: 'float' object is not subscriptable